In [ ]:
%cd /content/drive/MyDrive/
# raw_data is imported from global config
root_dir='/content/drive/MyDrive/'  # ChangeThis
result_dir=root_dir+'grasp/featureEEGNet/result/'


/content/drive/MyDrive


In [ ]:
! pip install mne==0.19.2;
! pip install torch;

     |████████████████████████████████| 6.4MB 3.6MB/s 


In [ ]:
import torch
from torch.nn import MSELoss, L1Loss
from torch.optim import SGD, Adagrad
import numpy as np
from torch.utils.data import Dataset, DataLoader
from grasp.featureEEGNet.model import model_simplify
from grasp.featureEEGNet.utils import evaluate, train
from grasp.utils import SEEGDataset,read_fbanddata,rawData2
from grasp.config import root_dir
import matplotlib.pyplot as plt


In [ ]:
# reload function
import sys, importlib
importlib.reload(sys.modules['grasp.featureEEGNet.model'])
from grasp.featureEEGNet.model import model_simplify
#importlib.reload(sys.modules['grasp.utils'])
#from grasp.utils import rawData2

In [ ]:
# test importing variable
from grasp.config import data_dir
data_dir

'/content/drive/MyDrive/data/'

In [ ]:
# test reloading function
import inspect as i
import sys
#sys.stdout.write(i.getsource(model_simplify));

In [ ]:
enable_cuda = torch.cuda.is_available()
print('GPU computing: ', enable_cuda)

GPU computing:  True


In [ ]:
%%capture
# suppress the output
sampling_rate=1000
#traindata, valdata, testdata = rawData2('raw',activeChannels,move2=False)  # (chns, 15000/15001, 118) (channels, time, trials)
traindata, valdata, testdata = rawData2('band','all',move2=True)
traindata = traindata.transpose(2, 0, 1)  # (118, 20, 15000) (trials,channels,  time)
valdata = valdata.transpose(2, 0, 1) # (8, 20, 15000)
testdata = testdata.transpose(2, 0, 1)  # (8, 20, 15000)
trainx, trainy = traindata[:, :-2, :], traindata[:, -2, :] #-2 is real force, -1 is target
valx, valy = valdata[:, :-2, :], valdata[:, -2, :]
testx, testy = testdata[:, :-2, :], testdata[:, -2, :]

dataset_train = SEEGDataset(trainx, trainy)
dataset_val = SEEGDataset(valx, valy)
dataset_test = SEEGDataset(testx, testy)

In [ ]:
train_loader = DataLoader(dataset=dataset_train, batch_size=1, shuffle=True, pin_memory=False)
val_loader = DataLoader(dataset=dataset_val, batch_size=1, pin_memory=False)
test_loader = DataLoader(dataset=dataset_test, batch_size=1, pin_memory=False)

In [ ]:
#wind=1000
#stride=500
wind=1000 
stride=1000
channlNum=trainx.shape[1]
kernLength1=wind//2
kernLength2=wind//2

epochs = 100
learning_rate = 0.0001

#model=torch.load(resultdir+'model_250.pth')
net = model_simplify(channlNum,kernLength1,kernLength2)
#criterion = MSELoss().float()
criterion = MSELoss()
# Adam can't even descend the training error, but hold still !!! Don't understand this.
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate) #, weight_decay=1e-4)

if(enable_cuda):
	net.cuda()
 
#checkpoint = torch.load(result_dir+'checkpoint35.pth')
#net.load_state_dict(checkpoint['state_dict'])
#optimizer.load_state_dict(checkpoint['optimizer'])

In [ ]:
def train(epoch,model,dataloader,optimizer,criterion,wind, stride,resultdir,plot='plot'):
    model.train()
    trainloses = []
    for i, (trainx, trainy) in enumerate(dataloader):
        print("Trial "+str(i))
        trainx=torch.squeeze(trainx)  # torch.Size([19, 15000])
        trainy=torch.squeeze(trainy) # torch.Size([15000])
        idx=0
        T=trainx.shape[1] # 15000
        # x.shape: torch.Size([19, 27, 1000])
        # y.shape: torch.Size([27, 1000])
        while idx<T-wind-stride:
            if idx==0:
                x=trainx[:,idx:wind] #torch.Size([19, 1000])
                x=torch.unsqueeze(x,1) #torch.Size([19, 1, 1000])
                y = trainy[idx:wind]
                y=np.expand_dims(y,axis=0)
                #y=torch.unsqueeze(y,0) #torch.Size([1, 1000])
                idx=idx+stride #500
            else:
                tmp=trainx[:,idx:idx+wind] #torch.Size([19, 1000])
                tmp=torch.unsqueeze(tmp,1) #torch.Size([19, 1, 1000])
                x=torch.cat((x,tmp),1)
                tmp2=trainy[idx:idx+wind]
                tmp2=np.expand_dims(tmp2,axis=0)
                #tmp2 = torch.unsqueeze(tmp2, 0)
                y=np.concatenate((y,tmp2),0)
                idx=idx+stride

        target=y[:,-1] # target is the last value
        batch_size=x.shape[1]
        x=x.permute(1,0,2)
        x=np.expand_dims(x,axis=1)
        
        if (enable_cuda):
            x = torch.from_numpy(x).float().cuda()
            target = torch.from_numpy(target).cuda()

        optimizer.zero_grad()
        pred = model(x)
        ls = criterion(torch.squeeze(pred), target.float())
        lose = ls.item()
        trainloses.append(lose)
        ls.backward()
        optimizer.step()
        with open(resultdir+"trainlose.txt", "a") as f:
            f.write(str(lose) + "\n")
    trainloseavg = sum(trainloses) / len(trainloses)

    print(f'epoch: {epoch:3} loss: {trainloseavg:10.8f}')


In [ ]:
def evaluate(epoch,model,dataloader,criterion,wind,stride,resultdir,plot='plot'):
    model.eval()
    with torch.no_grad():
        testloses = []
        preds = []
        targets = []
        for i, (testx, testy) in enumerate(dataloader):
            print("Trial "+ str(i))
            testx = torch.squeeze(testx)  # torch.Size([19, 15000])
            testy = torch.squeeze(testy)  # torch.Size([15000])
            idx = 0
            T = testx.shape[1]  # 15000
            # x.shape: torch.Size([19, 27, 1000])
            # y.shape: torch.Size([27, 1000])
            while idx < T - wind - stride:
                if idx == 0:
                    x = testx[:, idx:wind]  # torch.Size([19, 1000])
                    x = torch.unsqueeze(x, 1)  # torch.Size([19, 1, 1000])
                    y = testy[idx:wind]
                    y = torch.unsqueeze(y, 0)  # torch.Size([1, 1000])
                    idx = idx + stride  # 500
                else:
                    tmp = testx[:, idx:idx + wind]  # torch.Size([19, 1000])
                    tmp = torch.unsqueeze(tmp, 1)  # torch.Size([19, 1, 1000])
                    x = torch.cat((x, tmp), 1)
                    tmp2 = testy[idx:idx + wind]
                    tmp2=np.expand_dims(tmp2,axis=0)
                    y=np.concatenate((y,tmp2),0)
                    idx = idx + stride
            target = y[:, -1]
            
            batch_size = x.shape[1]
            x = x.permute(1, 0, 2)
            x=np.expand_dims(x,axis=1)
            if (enable_cuda):
                x = torch.from_numpy(x).float().cuda()
                target = torch.from_numpy(target).cuda()

            pred = model(x)

            preds.append(pred.cpu().data.numpy())
            targets.append(target.cpu().data.numpy())

            ls = criterion(torch.squeeze(pred), target.float())
            testlose = ls.item()
            testloses.append(testlose)
        testlosesavg = sum(testloses) / len(testloses)
        with open(resultdir+"testlose.txt", "a") as f:
            f.write(str(testlosesavg) + "\n")

        #flat_predict = [item for sublist in preds for item in sublist]
        #flat_target = [item for sublist in targets for item in sublist]
        preds = np.concatenate(preds,axis=0)
        targets = np.concatenate(targets, axis=0)

        if plot=='plot': # plot/noplot
            fig, ax = plt.subplots(figsize=(6, 3))
            plt.ion()
            ax.clear()
            ax.plot(targets, color="orange")
            ax.plot(preds, 'g-', lw=3)
            #plt.show()
            #plt.pause(0.2)  # Note this correction
            figname = resultdir+"test_epoch_%d" % epoch
            fig.savefig(figname)
            plt.close(fig)
        return preds, targets


In [ ]:
for epoch in range(0,epochs):
    #train(model,train_loader,'plot')
    print("------ epoch " + str(epoch) + " -----")
    train(epoch,net,train_loader,optimizer,criterion,wind,stride,result_dir, plot='plot')
    if epoch % 1 == 0:
        print("Evaluate")
        #_ ,_ = evaluate(model,test_loader,'plot')
        _,_ = evaluate(epoch,net,val_loader,criterion,wind,stride,result_dir, plot='plot')
    if epoch % 5 == 0:
        state = {
            'state_dict': net.state_dict(),
            'optimizer': optimizer.state_dict(),
        }
        savepath = result_dir+'checkpoint'+str(epoch)+'.pth'
        torch.save(state, savepath)
        
        
    